# 0. Data Preparation
This notebook handles data cleaning and feature engineering for the city bike data. It assumes that city bike ride and station location data have already been downloaded and saved to `/data/raw/` by the separate scripts. 

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# Define data directories
RAW_DIR = Path("../data/raw")
CLEAN_DIR = Path("../data/clean")
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

### Load the data

In [2]:
dtypes = {'departure_id': str, 'departure_name': str, 
            'return_id': str, 'return_name': str}
bike_df = pd.read_csv(RAW_DIR / 'bike_rides.csv', dtype=dtypes, parse_dates=['departure', 'return'])
bike_df.head()

,departure,return,departure_id,departure_name,return_id,return_name,distance,duration
0,2020-03-23 06:09:44,2020-03-23 06:16:26,086,Kuusitie,111,Esterinportti,1747.0,401.0
1,2020-03-23 06:11:58,2020-03-23 06:26:31,026,Kamppi (M),010,Kasarmitori,1447.0,869.0
2,2020-03-23 06:16:29,2020-03-23 06:24:23,268,Porolahden koulu,254,Agnetankuja,1772.0,469.0
3,2020-03-23 06:33:53,2020-03-23 07:14:03,751,Vallipolku,106,Korppaanmäentie,7456.0,2406.0
4,2020-03-23 06:36:09,2020-03-23 07:04:10,062,Välimerenkatu,121,Vilhonvuorenkatu,7120.0,1679.0


In [3]:
station_df = pd.read_csv(RAW_DIR / 'stations.csv')
station_df = station_df.set_index('id')
# Add leading zeros to IDs
station_df.index = station_df.index.fillna(-1).astype(int).astype(str).str.zfill(3)  
station_df.head()

,name,lat,lon,capacity,source
id,,,,,
150,Töölönlahden puisto,60.174646,24.935186,24.0,HSL
161,Eteläesplanadi,60.167231,24.947466,34.0,HSL
162,Leppäsuonaukio,60.169573,24.926590,28.0,HSL
163,Lehtisaarentie,60.181813,24.850440,12.0,HSL
118,Fleminginkatu,60.188154,24.952916,22.0,HSL


### Data Cleaning

The city bike season typically runs from the beginning of April to the end of October. In 2020, the season started unusually early on March 23rd. To maintain consistency across seasons, rides from March 2020 are excluded from the analysis.

In [4]:
bike_df = bike_df[bike_df['departure'].dt.month != 3]

Check the percentage of missing values for each column in the ride dataset.

In [5]:
missing_counts = bike_df.isnull().sum()
missing_percent = 100 * missing_counts / len(bike_df)
missing_summary = pd.DataFrame({'count': missing_counts, 'percent': round(missing_percent, 3)})
print(missing_summary)

                 count  percent
departure           69    0.001
return              20    0.000
departure_id         0    0.000
departure_name       0    0.000
return_id           79    0.001
return_name         79    0.001
distance          8479    0.063
duration        209240    1.544


The duration column has the most missing values and requires further investigation to decide how to handle them.
The other columns have such a small percentage of missing data that removing the rows with missing values will have a negligible impact on the analysis.

####  Exploring the Missing Duration Values


In [6]:
bike_df[(bike_df['departure'].dt.month == 10) & (bike_df['departure'].dt.year == 2021) & (bike_df['duration'].notna())]

,departure,return,departure_id,departure_name,return_id,return_name,distance,duration


All rows in October 2021 are missing the duration values, while other months are unaffected. This indicates the missing values are not random, but likely due to a data collection error. Removing months worth of data could impact the analysis results, when exploring seasonal patters and comparing yearly data. 

Since the data contains both departure and return timestamps, the duration can be calculated using these columns. Check if the timestamps are consistent with the duration values.

In [7]:
bike_df['duration_calc'] = (
    pd.to_datetime(bike_df['return']) - pd.to_datetime(bike_df['departure'])
).dt.total_seconds()

bike_df['duration_diff'] = abs(bike_df['duration_calc'] - bike_df['duration'])

print(bike_df['duration_diff'].describe())

count    1.334518e+07
mean     2.849358e+02
std      1.058912e+04
min      0.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      5.000000e+00
max      4.319138e+06
Name: duration_diff, dtype: float64


Although the mean difference between the calculated and reported durations is large (≈5 minutes), this is skewed by a few extreme outliers (with a maximum of ≈56 days). However, 75% of the calculated values are within 5 seconds of the reported values.

In [8]:
bike_df.loc[bike_df['duration'].isna(), 'duration'] = bike_df.loc[bike_df['duration'].isna(), 'duration_calc']

print('Duration for missing values filled with calculated values:')
print(bike_df[(bike_df['departure'].dt.month == 10) & (bike_df['departure'].dt.year == 2021)]['duration'].describe())
print('Duration October of the other years:')
print(bike_df[(bike_df['departure'].dt.month == 10) & (bike_df['departure'].dt.year != 2021)]['duration'].describe())

Duration for missing values filled with calculated values:
count    2.092400e+05
mean     1.611257e+03
std      1.666816e+04
min     -3.205000e+03
25%      3.280000e+02
50%      5.540000e+02
75%      9.430000e+02
max      2.499520e+06
Name: duration, dtype: float64
Duration October of the other years:
count    8.466140e+05
mean     1.021855e+03
std      1.107884e+04
min      0.000000e+00
25%      3.200000e+02
50%      5.450000e+02
75%      9.260000e+02
max      2.914721e+06
Name: duration, dtype: float64


The calculated duration values for the missing October 2021 data appear consistent with the data from October in other years. The median and 25 / 75th percentiles align closely, suggesting that the calculated values are reliable. However, both sets contain extreme outliers, which should be removed.

#### Remove the rows with missing values

In [9]:
# Drop rows with missing values and temporary duration columns
bike_df = bike_df.dropna().drop(columns=['duration_calc', 'duration_diff'])

# Convert seconds to minutes
bike_df['duration'] = bike_df['duration'] / 60  
bike_df.describe()

print(f'Percentage of rides that are over 5 hours: {round(len(bike_df[bike_df.duration > 5 * 60]) / len(bike_df), 3)}')

Percentage of rides that are over 5 hours: 0.003


From 2020 to 2023, the bike pass allowed free rides up to 30 minutes, with a charge of 1 euro for every additional 30 minutes, up to a total of 5 hours. After 5 hours, a delay fee of 80 euros applies, plus 9 euros for each additional 30 minutes. In 2024, the free ride period was extended to one hour, while the rest of the pricing structure remained the same.

The pricing incentivizes the users to utilize the bikes for short rides and maximizes the availability of the bikes. Only 0.3% of rides last over 5 hours, which is beyond the intended duration. Therefore, rows with a duration longer than 5 hours are removed. The activation and return of the bike require time, and thus rows with a duration under 1 minute are also removed, as these are likely errors or cases where the user activates and returns the bike to the station without riding it.

Rides where the distance is under 50 meters are also removed, for the same reason as rides under 1 minute. Rides with a distance over 15 km are also removed, as the bikes are designed for short rides, and the majority of longer rides are assumed to be errors.

In [10]:
bike_df = bike_df[(bike_df['duration'] <= 5 * 60) & (bike_df['duration'] > 1) & (bike_df['distance'] > 50) & (bike_df['distance'] < 15000)]
bike_df.reset_index(drop=True, inplace=True)
bike_df[['distance', 'duration']].describe()

,distance,duration
count,1.276228e+07,1.276228e+07
mean,2.485129e+03,1.387775e+01
std,1.777973e+03,1.337869e+01
min,5.100000e+01,1.016667e+00
25%,1.181000e+03,6.500000e+00
50%,2.004000e+03,1.085000e+01
75%,3.318000e+03,1.775000e+01
max,1.499800e+04,3.000000e+02


### Feature engineering

To support temporal analysis of bike usage patterns, we extract features such as `hour`, `month`, `year`, `weekday`, and `is_weekend` from the `departure` timestamp.
- `hour`: Captures daily patterns and helps identify peak usage times. 
- `weekday`: Helps detect weekly trends in ride behavior, such as workweek commuting and weekend leisure.
- `is_weekend`: A boolean flag indicating whether the ride occurred on a weekend, useful for separating weekday and weekend usage patterns.
- `month` and `year`: Useful for analyzing seasonal trends and enabling multi-year comparisons.


We also merge station location and capacity information into the dataset to enable spatial analysis and explore how geography impacts bike usage.
- `lat` and `lon`: Latitude and longitude coordinates that enable geospatial analysis and mapping. These help identify patterns in demand by location and station clustering.
- `capacity`: The maximum number of designated docking spots at a station. While this is the official station size, bikes can still be returned even when a station is full by locking them to existing bikes. As a result, actual usage can exceed the stated capacity.


In [11]:
def merge_station_info(bike_df, station_df, station_type):
    hsl_stations = station_df[station_df['source'] == 'HSL']
    osm_stations = station_df[station_df['source'] == 'OSM']

    # Merge HSL station info
    bike_df = bike_df.merge(
        hsl_stations[['lat', 'lon', 'capacity']].add_prefix(f'{station_type}_'),
        left_on=f'{station_type}_id',
        right_index=True,
        how='left'
    )
    # Merge OSM station info as separate columns
    bike_df = bike_df.merge(
        osm_stations[['lat', 'lon', 'capacity', 'name']].add_prefix(f'OSM_{station_type}_'),
        left_on=f'{station_type}_name',
        right_on=f'OSM_{station_type}_name',
        how='left'
    )

    # Fill missing values from OSM
    for col in ['lat', 'lon', 'capacity']:
        bike_df[f'{station_type}_{col}'] = bike_df[f'{station_type}_{col}'].fillna(bike_df[f'OSM_{station_type}_{col}'])

    # Drop the OSM columns
    bike_df = bike_df.drop(columns=[
        f'OSM_{station_type}_name',
        f'OSM_{station_type}_lat',
        f'OSM_{station_type}_lon',
        f'OSM_{station_type}_capacity'
    ])

    return bike_df

In [12]:
# Temporal features
bike_df['hour'] = bike_df['departure'].dt.hour
bike_df['month'] = bike_df['departure'].dt.month
bike_df['year'] = bike_df['departure'].dt.year
bike_df['weekday'] = bike_df['departure'].dt.weekday
bike_df['is_weekend'] = bike_df['weekday'] >= 5

# Merge with station info
bike_df = merge_station_info(bike_df, station_df, station_type='departure')
bike_df = merge_station_info(bike_df, station_df, station_type='return')

bike_df.head()

,departure,return,departure_id,departure_name,return_id,return_name,distance,duration,hour,month,year,weekday,is_weekend,departure_lat,departure_lon,departure_capacity,return_lat,return_lon,return_capacity
0,2020-04-01 00:04:08,2020-04-01 00:21:27,062,Välimerenkatu,062,Välimerenkatu,999.0,17.300000,0,4,2020,2,False,60.159296,24.916241,16.0,60.159296,24.916241,16.0
1,2020-04-01 00:12:31,2020-04-01 00:21:34,149,Toinen linja,016,Liisanpuistikko,2372.0,8.966667,0,4,2020,2,False,60.183164,24.944195,22.0,60.174140,24.962084,17.0
2,2020-04-01 00:16:46,2020-04-01 00:46:09,118,Fleminginkatu,105,Tilkantori,4299.0,18.016667,0,4,2020,2,False,60.188154,24.952916,22.0,60.202839,24.893476,16.0
3,2020-04-01 00:19:29,2020-04-01 00:30:13,017,Varsapuistikko,013,Merisotilaantori,1923.0,10.650000,0,4,2020,2,False,60.173103,24.949519,28.0,60.167870,24.975534,24.0
4,2020-04-01 00:22:32,2020-04-01 00:27:29,030,Itämerentori,067,Perämiehenkatu,1376.0,4.866667,0,4,2020,2,False,60.163531,24.914517,40.0,60.160088,24.934066,16.0


### Save cleaned data

In [13]:
bike_df.to_csv(CLEAN_DIR / 'bike_rides_cleaned.csv', index=False)

### Data Cleaning Summary
- Calculated missing duration values using departure and return timestamps.
- Removed data from March 2020 to maintain consistency in the city bike season.
- Removed rows with missing values.
- Filtered out rides with unrealistic durations and distances.
- Added temporal features: `hour`, `month`, `year`, `weekday`, and `is_weekend`.
- Merged station location information and capacity.